**This function is used to extract tbe label value of the string.**

As all the inputs are nested links in wiki data so its necessary to extaract the label using Web scraping.
Used BeautifulSoup

In [120]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pprint
import requests
# Defined a function to convert links to labels to put in text 
def ConverterFxn(input_Values):
  input_Values = input_Values.replace('entity','wiki')
  html = urlopen(input_Values)
  bsh = BeautifulSoup(html.read(), 'html.parser')
  x = bsh.h1
  x = str(x)
  # Converting link to usable form as extracted links are nested <Label type> and cant be used to web scrape
  Lidx = x.find('</span>')
  Iidx = x.find('wikibase-title-label') + 22
  New_string = ""
  for i in range(Iidx,Lidx):
    New_string = New_string + x[i]
  return(New_string)

Searching the required url from the input data

In [121]:
import pprint
from pydoc import describe
from re import S
import requests
# To Get Search Result we search the query o wiki data search box utilise the most relevant search
API_ENDPOINT = "https://www.wikidata.org/w/api.php"
query = input('Search Here ')

params = {
    'action': 'wbsearchentities',
    'format': 'json',
    'language': 'en',
    'search': query
}

r = requests.get(API_ENDPOINT, params = params)
Data = r.json()['search'][0]
print(r.json()['search'][0])

Search Here Microsoft
{'id': 'Q2283', 'title': 'Q2283', 'pageid': 3223, 'display': {'label': {'value': 'Microsoft', 'language': 'en'}, 'description': {'value': 'American multinational technology corporation', 'language': 'en'}}, 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q2283', 'concepturi': 'http://www.wikidata.org/entity/Q2283', 'label': 'Microsoft', 'description': 'American multinational technology corporation', 'match': {'type': 'label', 'language': 'en', 'text': 'Microsoft'}}


**Checking if the Data (initial) is working**

In [122]:
# Checking the functionality of link of page from which we will extracct data
print(Data['concepturi'])

http://www.wikidata.org/entity/Q2283


Installing qwikidata api

In [123]:
!pip3 install qwikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Installing mkwikidata

In [124]:
!pip3 install mkwikidata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Code to make/edit optimised queries for different links and instances**

In [125]:
import pprint
import mkwikidata

import requests

query2 = '''
  { ?article  schema:about       ?item ;
              schema:inLanguage  "en" ;
              schema:isPartOf    <https://en.wikipedia.org/>
              '''              
query3 = f'''          
    FILTER ( ?item = <{Data['concepturi']}> )
    '''
query4 = ''' 
   OPTIONAL
      { ?item  wdt:P735  ?Name }
   OPTIONAL
      { ?item  wdt:P1448  ?Company_Official_name }
   OPTIONAL
      { ?item  wdt:P103  ?Company_Industry }
   OPTIONAL
      { ?item  wdt:P34  ?Instance }
   OPTIONAL
      { ?item  wdt:P169  ?Company_chief_executive_officer }
   OPTIONAL
      { ?item  wdt:P17  ?Company_Country_Origin }
   OPTIONAL
      { ?item  wdt:P571  ?Comany_Inception_date }
   OPTIONAL
      { ?item  wdt:P112  ?Company_foudner }
   OPTIONAL
      { ?item  wdt:P127  ?Company_Owned_By }
   OPTIONAL
      { ?item  wdt:P159  ?Company_HeadQuarter_Location }
   OPTIONAL
      { ?item  wdt:P1128  ?Company_Employee_count }
   OPTIONAL
      { ?item  wdt:P8687  ?Social_Media_Followers }
   OPTIONAL
      { ?item  wdt:P166   ?Company_Award_Received }
   OPTIONAL
      { ?item  wdt:P355  ?Company_Subsidiary }
   
    SERVICE wikibase:label
      { bd:serviceParam
                  wikibase:language  "en"
      }
  }
GROUP BY ?item ?itemLabel ?itemDescription
'''


url = 'https://query.wikidata.org/sparql'
query1 = f'''
PREFIX  schema: <http://schema.org/>
PREFIX  bd:   <http://www.bigdata.com/rdf#>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX  wikibase: <http://wikiba.se/ontology#>

SELECT DISTINCT  ?item ?itemLabel ?itemDescription (SAMPLE(?Name) AS ?NameSample) (SAMPLE(?Company_Official_name) AS ?Company_Official_nameSample) (SAMPLE(?Company_Industry) AS ?Company_IndustrySample) (SAMPLE(?Instance) AS ?InstanceSample) (SAMPLE(?Company_chief_executive_officer) AS ?Company_chief_executive_officer) (SAMPLE(?Company_Country_Origin) AS ?Company_Country_OriginSample) (SAMPLE(?Comany_Inception_date) AS ?Comany_Inception_dateSample) (SAMPLE(?Company_foudner) AS ?Company_foudnerSample) (SAMPLE(?Company_Owned_By) AS ?Company_Owned_BySample) (SAMPLE(?Company_HeadQuarter_Location) AS ?Company_HeadQuarter_LocationSample) (SAMPLE(?Company_Employee_count) AS ?Company_Employee_countSample) (SAMPLE(?Social_Media_Followers) AS ?Social_Media_FollowersSample) (SAMPLE(?Company_Award_Received) AS ?Company_Award_ReceivedSample) (SAMPLE(?Company_Subsidiary) AS ?Company_SubsidiarySample)

WHERE

'''
query = query1 + query2 + query3 + query4


r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

**Checking if generated query is in perfect form**

In [126]:
print(query)


PREFIX  schema: <http://schema.org/>
PREFIX  bd:   <http://www.bigdata.com/rdf#>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX  wikibase: <http://wikiba.se/ontology#>

SELECT DISTINCT  ?item ?itemLabel ?itemDescription (SAMPLE(?Name) AS ?NameSample) (SAMPLE(?Company_Official_name) AS ?Company_Official_nameSample) (SAMPLE(?Company_Industry) AS ?Company_IndustrySample) (SAMPLE(?Instance) AS ?InstanceSample) (SAMPLE(?Company_chief_executive_officer) AS ?Company_chief_executive_officer) (SAMPLE(?Company_Country_Origin) AS ?Company_Country_OriginSample) (SAMPLE(?Comany_Inception_date) AS ?Comany_Inception_dateSample) (SAMPLE(?Company_foudner) AS ?Company_foudnerSample) (SAMPLE(?Company_Owned_By) AS ?Company_Owned_BySample) (SAMPLE(?Company_HeadQuarter_Location) AS ?Company_HeadQuarter_LocationSample) (SAMPLE(?Company_Employee_count) AS ?Company_Employee_countSample) (SAMPLE(?Social_Media_Followers) AS ?Social_Media_FollowersSample) (SAMPLE(?Company_Award_Received) AS ?Company

**Checking extracted data extracted with help of the query**

In [127]:
pprint.pprint(data)

{'head': {'vars': ['item',
                   'itemLabel',
                   'itemDescription',
                   'NameSample',
                   'Company_Official_nameSample',
                   'Company_IndustrySample',
                   'InstanceSample',
                   'Company_chief_executive_officer',
                   'Company_Country_OriginSample',
                   'Comany_Inception_dateSample',
                   'Company_foudnerSample',
                   'Company_Owned_BySample',
                   'Company_HeadQuarter_LocationSample',
                   'Company_Employee_countSample',
                   'Social_Media_FollowersSample',
                   'Company_Award_ReceivedSample',
                   'Company_SubsidiarySample']},
 'results': {'bindings': [{'Comany_Inception_dateSample': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                                                           'type': 'literal',
                                         

**Checking nested dictionary's functionality**

In [128]:
print(data['head']['vars'][8])

Company_Country_OriginSample


In [129]:
# print(data['results']['bindings'][0]['NameSample']['value'])

**Converting data to usable form: **

1.   Convertion of extracted data to usable variables.
2.   Converting the data links to labels that could be used in text
1.   Making sentences for the newly made variables.
2.   printing of the variables 

Note : If any of the existing steps fails the variable will be skipped hence either the data is not present or non-editable.

*All the variables that were required in all the groups are pre-labeled.*

In [138]:
Company = data['results']['bindings'][0]['itemLabel']['value']
output = ''
try:
  Description = data['results']['bindings'][0]['itemDescription']['value']
  Comp_Descriptio = ConverterFxn(Comp_Descriptio)
  Company_Descriptio = f'''It is {Description}'''
  print(Company_Descriptio)
  output = output + Company_Descriptio
except:
  pass  

try:
  Company_foudner = data['results']['bindings'][0]['Company_foudnerSample']['value']
  Company_founder = ConverterFxn(Company_foudner)
  Company_founder = f'''{Company} was mojorly found by {Company_founder}.'''
  print(Company_founder)
  output = output + Company_founder
except:
  pass

try:
  Comany_Inception_date = data['results']['bindings'][0]['Comany_Inception_dateSample']['value']
  Company_Inception = f'''It was incepted on {Comany_Inception_date}.'''
  print(Company_Inception)
  output = output + Company_Inception
except:
  pass

try:
  Company_Official_Name = data['results']['bindings'][0]['Company_Official_Name']['value']
  Company_Official_Name = ConverterFxn(Company_Official_Name)
  Company_Official_Name = f'''It has its official name as {Company_Official_name}.'''
  print(Company_Official_Name)
  output = output + Company_Official_Name
except:
  pass

try:
  Company_Official_name = data['results']['bindings'][0]['Company_Official_nameSample']['value']
  Company_Official_Name = f'''It has its official name as {Company_Official_name}.'''
  print(Company_Official_Name)
  output = output + Company_Official_Name
except:
  pass

try:
  Company_Industry = data['results']['bindings'][0]['Company_IndustrySample']['value']
  Comapny_Industry = ConverterFxn(Comapny_Industry)
  Comapny_Industry =  f'''It comes under the {Company_Industry}.'''
  print(Comapny_Industry)
  output = output + Comapny_Industry
except:
  pass

try:
  Instance = data['results']['bindings'][0]['InstanceSample']['value']
  Company_Instance = ConverterFxn(Company_Instance)
  Company_Instance =  f'''It is an instance of {Instance}'''
  print(Company_Instance)
  output = output + Company_Instance
except:
  pass

try:
  Company_Country_Origin = data['results']['bindings'][0]['Company_Country_OriginSample']['value']
  Company_Country_Origin = ConverterFxn(Company_Country_Origin)
  Company_Country_Origin = f'''It has its Origin country as {Company_Country_Origin}.'''
  print(Company_Country_Origin)
  output = output + Company_Country_Origin
except:
  pass

try:
  Company_chief_executive_officer = data['results']['bindings'][0]['Company_chief_executive_officer']['value']
  Company_chief_executive_officer = ConverterFxn(Company_chief_executive_officer)
  Company_chief_executive_officer = f'''It has its present chief executive officer as {Company_chief_executive_officer}.'''
  print(Company_chief_executive_officer)
  output = output + Company_chief_executive_officer
except:
  pass

try:
  Company_Owned_By = data['results']['bindings'][0]['Company_Owned_BySample']['value']
  Company_Owned_By = ConverterFxn(Company_Owned_By)
  Company_Owned_By = f'''It is owned by {Company_Owned_By}.'''
  print(Company_Owned_By)
  output = output + Company_Owned_By
except:
  pass

try:
  Company_Subsidiary = data['results']['bindings'][0]['Company_SubsidiarySample']['value']
  Company_Subsidiary = ConverterFxn(Company_Subsidiary)
  Company_Subsidary = f'''It has its major subsidiary {Company_Subsidiary}.''' 
  print(Company_Subsidary)
  output = output + Company_Subsidary
except:
  pass

try:
  Company_HeadQuarter_Location = NetWorth = data['results']['bindings'][0]['Company_HeadQuarter_LocationSample']['value']
  Company_HeadQuarter_Location = ConverterFxn(Company_HeadQuarter_Location)
  Company_HeadQuarter_Location = f'''It has its HeadQuarter location as {Company_HeadQuarter_Location}.'''
  print(Company_HeadQuarter_Location)
  output = output + Company_HeadQuarter_Location
except:
  pass

try:
  Company_Employee_count = NetWorth = data['results']['bindings'][0]['Company_Employee_countSample']['value']
  Company_Employee_count = f'''It has {Company_Employee_count} employee.'''
  print(Company_Employee_count)
  output = output + Company_Employee_count
except:
  pass

try:
  Company_Social_Media = NetWorth = data['results']['bindings'][0]['Social_Media_FollowersSample']['value']
  Company_Social_Media = f'''It has {Company_Social_Media} number of social media followers.'''
  print(Company_Social_Media)
  output = output + Company_Social_Media
except:
  pass

try:
  Company_Award_Received = NetWorth = data['results']['bindings'][0]['Company_Award_ReceivedSample']['value']
  Company_Award_Received = ConverterFxn(Company_Award_Received)
  Company_Award_Received = f'''It has received the {Company_Award_Received}.'''
  print(Company_Award_Received)
  output = output + Company_Award_Received
except:
  pass

Microsoft was mojorly found by Bill Gates.
It was incepted on 1976-11-26T00:00:00Z.
It has its official name as Microsoft Corporation.
It has its Origin country as United States of America.
It has its present chief executive officer as Satya Nadella.
It is owned by Bill Gates.
It has its major subsidiary GitHub.
It has its HeadQuarter location as Redmond.
It has 181000 employee.
It has 788000 number of social media followers.
It has received the BigBrotherAwards.


In [136]:
pprint.pprint(output)

('Microsoft was mojorly found by Bill Gates.Microsoft was incepted on '
 '1976-11-26T00:00:00Z.Microsoft has its official name as Microsoft '
 'Corporation.Microsoft has its Origin country as United States of '
 'America.Microsoft has its present chief executive officer as Satya '
 'Nadella.Microsoft is owned by Bill Gates.Microsoft has its major subsidiary '
 'GitHub.Microsoft has its HeadQuarter location as Redmond.Microsoft has '
 '181000 employee.Microsoft has 788000 number of social media '
 'followers.Microsoft has received the BigBrotherAwards.')
